In [1]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
background_path = 'Background_clean.csv'
log_path = 'Logs_clean.csv'

background_df = pd.read_csv(background_path)
log_df = pd.read_csv(log_path)

In [3]:
background_df.head()

,index,user_id,gender,age_group,LSM_group1,share_your_home_with,including_yourself_how_many_people_do_you_share_your_home_with,do_to_try_to_stay_healthy,drives_you_to_have_a_healthy_lifestyle,eat_or_drink,steps_to_eat_drink_more_healthily,how_regularly_do_you_exercise,exercise_do_you_enjoy,do_to_help_you_sleep,before_you_go_to_sleep
0,1,3.892140e+08,Female,25-34,High,"[""Children over 18 years old""]",4.0,"[""Stay hydrated""]","[""To make the most out of/enjoy life""]","[""None of the above""]","[""Eating out at places that have healthy optio...",3+ times a week,"[""Running"", ""Yoga/Pilates""]","[""Have a bath"", ""Have a malted milk drink""]","[""Browsing on my cell phone / tablet"", ""Caffei..."
1,3,0.000000e+00,Female,45+,High,"[""Partner / spouse""]",1,"[""Drink lots of water / fluids"", ""Eat healthil...","[""To manage a health / medical condition"", ""Pr...","[""Flexitarian: eat meat but control the amount...","[""Intermittent fasting""]",Never,NaN,"[""Have herbal tea"", ""Have a bath""]","[""Caffeine"", ""Heavy meal"", ""Other""]"
2,4,2.647339e+09,Male,25-34,High,"[""Parents"", ""Other family members""]",4.0,"[""Taking time to relax"", ""Eat healthily"", ""Dri...","[""To make the most of treats / indulgent momen...","[""Low sugar"", ""Low salt"", ""Clean eating: eat m...","[""Trying to have a variety of colors on my pla...",3+ times a week,"[""Walking"", ""Cycling"", ""Hiking""]","[""Have warm milk"", ""Other""]","[""Heavy meal"", ""Caffeine""]"
3,5,9.512227e+08,Female,45+,High,"[""Partner / spouse""]",3.0,"[""Eat healthily"", ""Exercise regularly"", ""Make ...","[""To improve my sleep"", ""To manage a health / ...","[""Low salt""]","[""Cooking more meals at home""]",Twice a week,"[""Hiking""]","[""No, none of these""]","[""Caffeine""]"
4,6,1.056650e+09,Male,35-44,Medium,"[""Children under 18 years old"", ""Partner / spo...",5+,"[""Make sure I get enough sleep"", ""Taking time ...","[""To make the most out of/enjoy life"", ""To imp...","[""None of the above""]","[""Cooking more meals at home""]",Never,NaN,"[""No, none of these""]","[""Caffeine""]"


In [4]:
log_df.head()

,submission_id,user_id,location_name,submission_timestamp,where_are_you_eating_1,you_with,where_are_you_eating_2,you_got_your_food_from,while_eating_your_food,who_prepared_the_food,level2dish,reasons_you_chose_this_food,other_food_you_consider_having,describes_the_food_you_have
0,1,3.892140e+08,South Africa,2019-01-31 15:11:16.447589,At school / work,"[""Alone""]",At school / work,I had it at home / brought it from home,"[""Working / studying""]",I prepared it myself from scratch,"[""Pork""]","[""I like the taste""]","[""Nothing""]",Lunch
1,4,2.647339e+09,South Africa,2019-01-24 18:30:12.830288,At my home,"[""Alone""]",At my home,I had it at home / brought it from home,"[""Watching TV"", ""Browsing social media""]",Someone else prepared it,"[""Other potatoes/ sweet potatoes"", ""Bread/ rol...","[""It's a healthy choice"", ""Cheap""]","[""Meat""]",Dinner
2,6,1.056650e+09,South Africa,2019-08-22 15:13:50.876941,On the go / outdoors,"[""My spouse/partner"", ""My children""]",On the go / outdoors,Supermarket / Hypermarket / Grocery Store,"[""Spending time with family""]",It was premade / ready to eat,"[""Biscuits""]","[""Quick"", ""Kids/ family like it""]","[""Nothing""]",Snack
3,7,2.771745e+09,South Africa,2019-05-24 16:21:32.007399,At my home,"[""Alone""]",At my home,NaN,"[""Talking to someone"", ""Spending time with fri...",Someone else prepared it,"[""Cake""]","[""I like the taste""]","[""Nothing""]",Snack
4,8,3.427770e+09,South Africa,2019-02-13 04:02:58.491098,At my home,"[""Alone""]",At my home,I had it at home / brought it from home,"[""Watching TV""]",I prepared it myself from scratch,"[""Bread/ roll/ baguette"", ""Instant noodles""]","[""I like the taste""]","[""Nothing""]",Snack


In [5]:
def get_index_in_set(set_val, item):
    index = 0
    return_index = -1
    for set_item in set_val:
        if set_item == item:
            return_index = index
        index += 1
    return return_index

In [6]:
def list_to_multi_hot(pd_series_list):
    unique_values = set()
    for _, value_list in pd_series_list.iteritems():
        if pd.notnull(value_list):
            for value in ast.literal_eval(value_list):
                unique_values.add(value)
    unique_count = len(unique_values)
    multi_hot = []
    for _, value_list in pd_series_list.iteritems():
        multi_hot_item = [0] * unique_count
        if pd.notnull(value_list):
            for value in ast.literal_eval(value_list):
                index = get_index_in_set(unique_values, value)
                if index != -1:
                    multi_hot_item[index] = 1
        multi_hot.append(multi_hot_item)
    multi_hot = np.array(multi_hot)
    multi_hot = pd.DataFrame(multi_hot)
    multi_hot.columns = [pd_series_list.name + '_' + choice for choice in list(unique_values)]
    return multi_hot

In [7]:
background_df.iloc[325:330, :].exercise_do_you_enjoy

325    ["Football"]
326             NaN
327             NaN
328     ["Running"]
329             NaN
Name: exercise_do_you_enjoy, dtype: object

In [8]:
'eat_or_drink'
'exercise_do_you_enjoy'
list_to_multi_hot(background_df.iloc[325:330, :].exercise_do_you_enjoy)

,exercise_do_you_enjoy_Running,exercise_do_you_enjoy_Football
0,0,1
1,0,0
2,0,0
3,1,0
4,0,0


In [9]:
list_columns = ['share_your_home_with', 'do_to_try_to_stay_healthy',
               'drives_you_to_have_a_healthy_lifestyle', 'eat_or_drink',
               'steps_to_eat_drink_more_healthily', 'exercise_do_you_enjoy',
               'do_to_help_you_sleep', 'before_you_go_to_sleep']

for col_name in list_columns:
    multi_hot = list_to_multi_hot(background_df[col_name])
    del background_df[col_name]
    background_df = background_df.join(multi_hot)

In [10]:
background_df.head()

,index,user_id,gender,age_group,LSM_group1,including_yourself_how_many_people_do_you_share_your_home_with,how_regularly_do_you_exercise,share_your_home_with_Friends / Flatmates,share_your_home_with_I live alone,share_your_home_with_Other family members,...,do_to_help_you_sleep_Have a bath,before_you_go_to_sleep_Checking emails,before_you_go_to_sleep_Heavy meal,before_you_go_to_sleep_Caffeine,"before_you_go_to_sleep_No, none of these",before_you_go_to_sleep_Watching TV,before_you_go_to_sleep_Alcohol,before_you_go_to_sleep_Other,before_you_go_to_sleep_Browsing on my cell phone / tablet,before_you_go_to_sleep_Browsing on my mobile phone / tablet
0,1,3.892140e+08,Female,25-34,High,4.0,3+ times a week,0,0,0,...,1,0,0,1,0,1,1,0,1,0
1,3,0.000000e+00,Female,45+,High,1,Never,0,0,0,...,1,0,1,1,0,0,0,1,0,0
2,4,2.647339e+09,Male,25-34,High,4.0,3+ times a week,0,0,1,...,0,0,1,1,0,0,0,0,0,0
3,5,9.512227e+08,Female,45+,High,3.0,Twice a week,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,6,1.056650e+09,Male,35-44,Medium,5+,Never,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [11]:
list_columns = ['you_with', 'while_eating_your_food', 'level2dish', 'reasons_you_chose_this_food',
               'other_food_you_consider_having']

for col_name in list_columns:
    multi_hot = list_to_multi_hot(log_df[col_name])
    del log_df[col_name]
    log_df = log_df.join(multi_hot)

In [12]:
log_df.head()

,submission_id,user_id,location_name,submission_timestamp,where_are_you_eating_1,where_are_you_eating_2,you_got_your_food_from,who_prepared_the_food,describes_the_food_you_have,you_with_Colleagues,...,other_food_you_consider_having_Rice / pasta / grains / pulses,other_food_you_consider_having_Savoury pastry / pie,other_food_you_consider_having_Healthy / meal replacement drinks,other_food_you_consider_having_Carbonated soft drink,other_food_you_consider_having_Eggs,other_food_you_consider_having_Other drinks,other_food_you_consider_having_Confectionery,other_food_you_consider_having_Other Dishes,other_food_you_consider_having_Desserts,other_food_you_consider_having_Hot tea
0,1,3.892140e+08,South Africa,2019-01-31 15:11:16.447589,At school / work,At school / work,I had it at home / brought it from home,I prepared it myself from scratch,Lunch,0,...,0,0,0,0,0,0,0,0,0,0
1,4,2.647339e+09,South Africa,2019-01-24 18:30:12.830288,At my home,At my home,I had it at home / brought it from home,Someone else prepared it,Dinner,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1.056650e+09,South Africa,2019-08-22 15:13:50.876941,On the go / outdoors,On the go / outdoors,Supermarket / Hypermarket / Grocery Store,It was premade / ready to eat,Snack,0,...,0,0,0,0,0,0,0,0,0,0
3,7,2.771745e+09,South Africa,2019-05-24 16:21:32.007399,At my home,At my home,NaN,Someone else prepared it,Snack,0,...,0,0,0,0,0,0,0,0,0,0
4,8,3.427770e+09,South Africa,2019-02-13 04:02:58.491098,At my home,At my home,I had it at home / brought it from home,I prepared it myself from scratch,Snack,0,...,0,0,0,0,0,0,0,0,0,0
